In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import cohen_kappa_score

In [2]:
base_dir = "vol06"
#names = ['berk', 'cansu', 'yagmur'] # vol02
#names = ['bugra', 'ecem', 'furkan', 'seyda'] # vol03
#names = ['ata', 'canberk', 'deniz', 'oya', 'yavuz'] # vol04
#names = ['aslı', 'begum', 'doga', 'utku', 'yigithan'] # vol05
names = ['berkecan', 'damla', 'eren', 'irmak', 'metehan'] # vol06
#names = ['arda', 'doga', 'ece', 'furkan', 'umutcan'] # vol07
#names = ['kerem', 'emre', 'bora', 'beyza', 'sema'] # vol08

In [3]:
#engagement_levels = [0, 1] # Disengaged - Engaged
engagement_levels = [-2, -1, 0, 1, 2]
#engagement_levels = ["h-diseng", "diseng", "moderate", "eng", "h-eng", "s-eng"]

In [4]:
from pathlib import Path

score_frames_list = []
filename_list = []
score_dirs = [os.path.join(base_dir, n) for n in names]
print(score_dirs)
files = Path(score_dirs[0]).glob('*.csv')
for file in files:
    filename = os.path.basename(file) #0.csv
    filename_list.append(filename)
    level_array = []
    for s in score_dirs:
        df = pd.read_csv(os.path.join(s, filename), sep=',', header=None)
        level_array.append(df.values[9:,1].astype(float).astype(int))

    score_frames = pd.DataFrame(np.transpose(level_array), columns=names)
    score_frames_list.append(score_frames)

['vol06\\berkecan', 'vol06\\damla', 'vol06\\eren', 'vol06\\irmak', 'vol06\\metehan']


In [5]:
score_frames_list[2].head()

,berkecan,damla,eren,irmak,metehan
0,2,1,0,7,56
1,16,2,0,54,88
2,-12,2,0,62,90
3,-21,2,0,82,90
4,-26,2,0,78,90


In [6]:
def checkLevel(l0):
    next_level = engagement_levels[0]
    if(l0 < -60):
        next_level = engagement_levels[0]
    elif(l0 >= -60 and l0 < -20):
        next_level = engagement_levels[1]
    elif(l0 > -20 and l0 <= 20):
        next_level = engagement_levels[2]
    elif(l0 > 20 and l0 <= 60):
        next_level = engagement_levels[3]
    elif(l0 > 60):
        next_level = engagement_levels[4]
    return next_level

"""
def checkLevel(l0):
    next_level = engagement_levels[0]
    if(l0 <= 0):
        next_level = engagement_levels[0]
    else:
        next_level = engagement_levels[1]
    return next_level
"""

'\ndef checkLevel(l0):\n    next_level = engagement_levels[0]\n    if(l0 <= 0):\n        next_level = engagement_levels[0]\n    else:\n        next_level = engagement_levels[1]\n    return next_level\n'

In [7]:
!mkdir {base_dir + "\\group\\levels"}
!mkdir {base_dir + "\\group\\scores"}

In [8]:
level_frames = pd.DataFrame(columns=names)
level_frames_list = []
i = 0
for score_frame in score_frames_list:
    for s in names:
        level_frames[s] = score_frame.apply(lambda row: checkLevel(row[s]), axis=1)

    
    level_frames['group'] = level_frames.min(axis=1)
    level_frames.to_csv(os.path.join(base_dir, "group/levels/%s" % filename_list[i]), index=False)
    score_frame.to_csv(os.path.join(base_dir, "group/scores/%s" % filename_list[i]), index=False)
    level_frames_list.append(level_frames)
    level_frames = pd.DataFrame(columns=names)
    i += 1

In [9]:
level_frames_list[0].head()
#print(len(level_frames_list[2]))

,berkecan,damla,eren,irmak,metehan,group
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0


## Calculate Kappa Score Between Engagement Levels

In [ ]:
# you can also check cohen kappa for two-rater agreement
k = cohen_kappa_score(level_frames[score_dirs[0]], level_frames[score_dirs[1]], weights='quadratic')
print(k)

In [ ]:
from statsmodels.stats import inter_rater as irr

for level_frame in level_frames_list:
    agg = irr.aggregate_raters(level_frame)
    print(irr.fleiss_kappa(agg[0], method='fleiss'))

## Read Level CSV and Determine Group Levels

In [ ]:
from pathlib import Path

level_frames_list = []
filename_list = []
files = Path("group/levels-binary/").glob('*.csv')
for file in files:  
    df = pd.read_csv(file, sep=',')
    df['group'] = df.min(axis=1)
    df.to_csv(file, index=False)

## Read CSV and Combine

In [10]:
from pathlib import Path

files = [f for f in Path("./vol06/group/levels").glob('*.csv')]
#files = sorted([int(f.stem) for f in Path("group\levels-new").glob('*.csv')])
print(len(files))
df = pd.read_csv(files[0])
df['file'] = files[0].stem
for i in range(len(files)):
    dfi = pd.read_csv(files[i], sep=',')
    dfi['file'] = files[i].stem
    df= pd.concat([df, dfi], ignore_index=False)

df['group'] = df.iloc[:, 1:4].min(axis=1)
df.to_csv("./vol06/group/all_levels.csv", index_label='frame')

10


In [13]:
df = pd.read_csv("../scores/vol06/group/all_levels.csv")

In [14]:
df['group'].value_counts()

 0    861
-1    816
-2    814
 1    724
 2    169
Name: group, dtype: int64

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
from tueplots import figsizes, fonts
from tueplots.constants.color import palettes

person = "berk"

fonts.neurips2021()
plt.rcParams["figure.dpi"] = 150
plt.rcParams["figure.figsize"] = (20,5)
fig = plt.figure()
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Frames')
ax.set_ylabel('Score')

ax.plot(df.loc[:1000,person])

plt.show()
#fig.savefig('fig/%s_level.png' % person)

# Read All Levels and Return Stats

In [15]:
for i in range(2,9):
    df = pd.read_csv("vol0%d/group/all_levels.csv" % i)
    display(df['group'].value_counts())

 2    2812
 1    1411
 3    1303
 0     707
-2     494
-1     242
Name: group, dtype: int64

-2    819
 1    792
-1    603
 0    532
 2    387
Name: group, dtype: int64

 0    1734
-1     889
 1     220
-2     196
 2     153
Name: group, dtype: int64

 0    1762
-1     874
 1     442
 2     433
-2     246
Name: group, dtype: int64

 0    861
-1    816
-2    814
 1    724
 2    169
Name: group, dtype: int64

 0    1516
 1    1175
 2     561
-1     218
-2     174
Name: group, dtype: int64

 1    942
 0    843
-1    719
-2    488
 2    157
Name: group, dtype: int64